In [1]:
import pandas as pd

In [5]:
df_en = pd.read_json('wiki_en.json')

In [2]:
df_ru = pd.read_json('wiki_ru.json')

Merge по изображениям

In [6]:
df_en_ru = df_en.merge(df_ru, on=['image_url'], how='left', indicator=True)
df_en_ru

,image_url,caption_description_x,caption_description_y,_merge
0,https://upload.wikimedia.org/wikipedia/commons...,"Nameplate of Gazeta Transilvaniei on June 14, ...",NaN,left_only
1,https://upload.wikimedia.org/wikipedia/commons...,A captive decoy,Подсадная птица,both
2,https://upload.wikimedia.org/wikipedia/commons...,The former Bus. 41 along Washington Street in...,NaN,left_only
3,https://upload.wikimedia.org/wikipedia/commons...,"Viktor Vekselberg, head of the Skolkovo Instit...",Президент Фонда «» Виктор Вексельберг,both
4,https://upload.wikimedia.org/wikipedia/commons...,Volunteer planting poppies,NaN,left_only
...,...,...,...,...
2349693,https://upload.wikimedia.org/wikipedia/commons...,Massachusetts Bay Transportation Authority 170...,NaN,left_only
2349694,https://upload.wikimedia.org/wikipedia/commons...,Fortified church,NaN,left_only
2349695,https://upload.wikimedia.org/wikipedia/commons...,Bisaldeo temple in 2016 with submerged courtyard,NaN,left_only
2349696,https://upload.wikimedia.org/wikipedia/commons...,Beyer in 2013,NaN,left_only


In [7]:
df_ru_en = df_ru.merge(df_en, on=['image_url'], how='left', indicator=True)
df_ru_en

,image_url,caption_description_x,caption_description_y,_merge
0,https://upload.wikimedia.org/wikipedia/commons...,Владислав Захаревич,NaN,left_only
1,https://upload.wikimedia.org/wikipedia/commons...,Зарисовка крупнейшего из Золотых ромбов курган...,Design of the larger of the Bush Barrow gold l...,both
2,https://upload.wikimedia.org/wikipedia/commons...,Трассовый моделизм. Прохождение моделями виража,NaN,left_only
3,https://upload.wikimedia.org/wikipedia/commons...,Спаривание,NaN,left_only
4,https://upload.wikimedia.org/wikipedia/commons...,Музей Дмитрия Бурылина,Industrial and art museum,both
...,...,...,...,...
449907,https://upload.wikimedia.org/wikipedia/commons...,Изображение мужской особи человека (самец вида...,NaN,left_only
449908,https://upload.wikimedia.org/wikipedia/commons...,Пасхальное украшение. ЖД станция в Норрчёпинге...,NaN,left_only
449909,https://upload.wikimedia.org/wikipedia/commons...,Вид на «Красный ». Осень 2017 г,NaN,left_only
449910,http://upload.wikimedia.org/wikipedia/commons/...,Максимилиан фон Родаковский во главе 13-го пол...,Rodakowski at the Battle of Custozza. (1908. O...,both


In [8]:
mask = (df_en_ru['_merge'] == 'left_only') #изображения, которые есть только в df_en
mask.value_counts()

True     2184123
False     165575
Name: _merge, dtype: int64

In [9]:
mask_ru_en = (df_ru_en['_merge'] == 'left_only') #изображения, которые есть только в df_ru
mask_ru_en.value_counts()

True     284337
False    165575
Name: _merge, dtype: int64

In [10]:
from tqdm import tqdm

In [11]:
df_to_translate_ru_en = df_ru_en[mask_ru_en]

In [12]:
df_to_translate_ru_en.index = list(range(df_to_translate_ru_en.shape[0]))

In [13]:
size = df_to_translate_ru_en.shape[0]

In [3]:
%pip install mtranslate

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3678 sha256=9ebce68823a58d13729741880cd3772e92688001460d175c6d8b1ebb106f341b
  Stored in directory: c:\users\dasha\appdata\local\pip\cache\wheels\ce\04\2c\04f74516aee24cdc301031f073f0397b4721a0271ee8ed073a
Successfully built mtranslate
Note: you may need to restart the kernel to use updated packages.


In [4]:
from mtranslate import translate

In [14]:
from joblib import delayed

In [15]:
from tqdm.auto import tqdm
from joblib import Parallel

class ProgressParallel(Parallel):
    def __init__(self, use_tqdm=True, total=None, *args, **kwargs):
        self._use_tqdm = use_tqdm
        self._total = total
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        with tqdm(disable=not self._use_tqdm, total=self._total) as self._pbar:
            return Parallel.__call__(self, *args, **kwargs)

    def print_progress(self):
        if self._total is None:
            self._pbar.total = self.n_dispatched_tasks
        self._pbar.n = self.n_completed_tasks
        self._pbar.refresh()

In [16]:
from itertools import chain

In [18]:
def translate_ru_en(i):
    result = []
    start = int(i * size / N)
    end = int((i + 1) * size / N) if i < N - 1 else size
    for index, row in df_to_translate_ru_en[start:end].iterrows():
        result.append((index, translate(row['caption_description_x'])))
    return result

In [16]:
import time

In [ ]:
N = 10
result = []
for i in range(size // 1000):
    df_i = df_to_translate_ru_en.iloc[1000 * i:1000 * (i + 1)]
    df_i.index = list(range(df_i.shape[0]))
    data = ProgressParallel(n_jobs=N, verbose=10)(delayed(translate_ru_en)(k) for k in list(range(N)))
    time.sleep(60)
    result = result + list(chain(*data))

0it [00:00, ?it/s]

[ProgressParallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


In [ ]:
df_result = pd.DataFrame(result)
df_result = df_result.rename(columns={'caption_reference_description' : 'caption_description'})
json = df_result.to_json(orient="records")
with open("wiki_translate_ru_en.json", "w") as outfile: 
    outfile.write(json)

In [19]:
N = 10
size = 1000
# df_to_translate_ru_en = df_to_translate_ru_en.iloc[0:10000]
data = ProgressParallel(n_jobs=N, verbose=10)(delayed(translate_ru_en)(i) for i in list(range(N)))

0it [00:00, ?it/s]

[ProgressParallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[ProgressParallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   34.7s finished


In [20]:
data

[[(0, 'Vladislav Zacharevich'),
  (1, 'Track modeling. Models passing a bend'),
  (2, 'Pairing'),
  (3, 'Wise Tsar Berendey. Sketch by Viktor Vasnetsov for the opera'),
  (4, 'Coat of arms of Stahl von Holstein'),
  (5, 'Monument to Ivanov in Tomsk'),
  (6, 'A gift to the tooth fairy from an eight year old girl'),
  (7, 'Faculty of Medicine, Seljuk University'),
  (8, "Taranov-Belozerov's hospice house (No. 28/10). year 2014"),
  (9, 'Seal of the Khan Wang of the Japanese country Na'),
  (10, 'Dean and Dan Caten as children'),
  (11, 'Former Ballpark station'),
  (12, 'Medal of the Portuguese Numismatic Society'),
  (13, 'Sassen (Eifel'),
  (14, 'Monument in Syktyvkar'),
  (15, 'Sysertsky pond'),
  (16, 'Yuri Smagin - played 361 matches for the club'),
  (17, 'Modernized according to project 1855.1 "AS-" in the Amur Bay'),
  (18, 'Sky through the smoke exit of a Tuvan yurt'),
  (19, 'The Beast of Gevaudan eats the corpses of its victims'),
  (20,
   'The Bikin River in the area of \u20

In [21]:
df_to_translate_ru_en['caption_description_x'].iloc[0:1000]

0                                    Владислав Захаревич
1        Трассовый моделизм. Прохождение моделями виража
2                                             Спаривание
3      Премудрый царь Берендей. Эскиз Виктора Васнецо...
4                             Герб Сталь-фон-Гольштейнов
                             ...                        
995                                     Улица Заморёнова
996       Мозаика «Святой » (Исаакиевский — левый придел
997    Ночная панорама Порту (вид со стороны Вила-Нов...
998                                   Меч лурской работы
999          Смеющийся Демокрит \nна картине Й. Мореелсе
Name: caption_description_x, Length: 1000, dtype: object